In [ ]:
# 2V25saC0pDFc8heT7wxsjigW3CF_32oQPdVZZP9wSMGd29FCa
# 2NqtkyBMF0KF99ofEU1fGz1pCJS_3XehxpoU47JGBiqY6tV7M

!pip install TTS openai-whisper pyngrok fastapi[all] nest_asyncio -q
!curl -s https://ngrok-agent.s3.amazonaws.com/ngrok.asc | sudo tee /etc/apt/trusted.gpg.d/ngrok.asc >/dev/null && echo "deb https://ngrok-agent.s3.amazonaws.com buster main" | sudo tee /etc/apt/sources.list.d/ngrok.list && sudo apt update && sudo apt install ngrok
!ngrok authtoken 2NqtkyBMF0KF99ofEU1fGz1pCJS_3XehxpoU47JGBiqY6tV7M
!sudo apt update && sudo apt upgrade && sudo apt install ffmpeg
!pip install --pre torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/nightly/cpu

deb https://ngrok-agent.s3.amazonaws.com buster main
Get:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:3 http://security.ubuntu.com/ubuntu jammy-security InRelease
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [495 kB]
Hit:5 https://ngrok-agent.s3.amazonaws.com buster InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:8 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:9 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Fetched 497 kB in 2s (234 kB/s)
Reading 

In [ ]:
import whisper
from fastapi import FastAPI, UploadFile, File, HTTPException, Body
from fastapi.middleware.cors import CORSMiddleware
from pydantic import BaseModel
import nest_asyncio
from pyngrok import ngrok
import uuid
import os
import uvicorn
import torch
from TTS.api import TTS
from fastapi.responses import FileResponse
import asyncio
import gc

device = 'mps' if torch.backends.mps.is_available() else 'cpu'

voice_model = whisper.load_model("large-v2").to(device)
tts = TTS('tts_models/en/jenny/jenny').to(device)
port=8888
gc.collect()

100%|██████████████████████████████████████| 2.87G/2.87G [00:24<00:00, 127MiB/s]


In [ ]:
app = FastAPI(title="SIH 2023 Backend")

app.add_middleware(
CORSMiddleware,
allow_origins=["*"],
allow_credentials=True,
allow_methods=["*"],
allow_headers=["*"],
)

class TTSModel(BaseModel):
    text: str
    emotion: str = "Cheerful & Professional"

@app.post("/transcribe/")
async def transcribe_audio(file: UploadFile = File(...)):
    audio_path = f"{uuid.uuid4()}.webm"
    with open(audio_path, "wb") as f:
        f.write(await file.read())
    result = voice_model.transcribe(whisper.pad_or_trim(whisper.load_audio(audio_path)))["text"]
    os.remove(audio_path)
    return {"text": result}

@app.post("/coqui-tts/")
async def coqui_tts(request: TTSModel = Body(...)):
    out = f"{uuid.uuid4()}.ogg"
    async def remove():
        loop = asyncio.get_event_loop()
        await loop.run_in_executor(None, lambda: os.remove(out))
    tts.tts_to_file(request.text, file_path=out, emotion=request.emotion)
    return FileResponse(out,headers={"Content-Disposition":f"attachment; filename={out}"},background=remove)

if __name__=="__main__":
    ngrok_tunnel = ngrok.connect(port)
    print('Public URL:', ngrok_tunnel.public_url)
    nest_asyncio.apply()
    uvicorn.run(app,port=port)